In [12]:
#%matplotlib widget

import requests
import time
import matplotlib.pyplot as plt
import re
from IPython.display import display, clear_output
import numpy as np
import scipy.spatial as spatial
from scipy.spatial import Delaunay
import binascii
from IPython.display import clear_output
from IPython import display

esp32_ip = '192.168.0.238'
esp32 = '192.168.4.1'


In [13]:
import struct
import math

# Constants
POINTS_PER_PACKET = 12
POINTS_SIZE = POINTS_PER_PACKET * 3  # 12 points * 3 bytes per point
PACKET_HEADER_SIZE = 11  # Size of the packet header
PACKET_SIZE = PACKET_HEADER_SIZE + POINTS_SIZE  # Total packet size

#PACKET_SIZE = 11 + POINTS_PER_PACKET * 3  # Adjust based on actual packet size
#PACKET_SIZE = 9 (header) + 12 * 3 (lidar points) + 1 (CRC) = 9 + 36 + 1 = 46 bytes

class LidarPoint:
    def __init__(self, distance, intensity):
        self.distance = distance
        self.intensity = intensity

class LidarPacket:
    def __init__(self, header, ver_len, speed, start_angle, points, end_angle, timestamp, crc):
        self.header = header
        self.ver_len = ver_len
        self.speed = speed
        self.start_angle = start_angle
        self.points = points
        self.end_angle = end_angle
        self.timestamp = timestamp
        self.crc = crc
        
def hex_to_binary(hex_string):
    return bytes.fromhex(hex_string)

def parse_lidar_data(data):
    # Check total data length
    if len(data) < 11 + POINTS_PER_PACKET * 3:
        print("Data length is insufficient. Length:", len(data))
        return None

    # Unpack header
    try:
        header, ver_len, speed, start_angle, end_angle, timestamp, crc = struct.unpack('<BBHHHHB', data[:11])
    except struct.error as e:
        print("Error unpacking header:", e)
        return None

    #print("Header unpacked:", header, ver_len, speed, start_angle, end_angle, timestamp, crc)

    # Process points data
    points_data = data[11:]  # Adjust the starting index based on your packet structure
    points = []
    for i in range(POINTS_PER_PACKET):
        point_slice = points_data[i * 3:(i + 1) * 3]
        if len(point_slice) == 3:
            distance, intensity = struct.unpack('<HB', point_slice)
            points.append(LidarPoint(distance, intensity))
            #print(f"Point {i}: Distance = {distance}, Intensity = {intensity}")
        else:
            print(f"Insufficient data for point {i}. Data length: {len(point_slice)}")

    return LidarPacket(header, ver_len, speed, start_angle, points, end_angle, timestamp, crc)

def cal_crc8(data):
    crc_table = [
  0x00, 0x4d, 0x9a, 0xd7, 0x79, 0x34, 0xe3, 0xae, 0xf2, 0xbf, 0x68, 0x25, 0x8b, 0xc6, 0x11, 0x5c,
  0xa9, 0xe4, 0x33,
  0x7e, 0xd0, 0x9d, 0x4a, 0x07, 0x5b, 0x16, 0xc1, 0x8c, 0x22, 0x6f, 0xb8, 0xf5, 0x1f, 0x52, 0x85,
  0xc8, 0x66, 0x2b,
  0xfc, 0xb1, 0xed, 0xa0, 0x77, 0x3a, 0x94, 0xd9, 0x0e, 0x43, 0xb6, 0xfb, 0x2c, 0x61, 0xcf, 0x82,
  0x55, 0x18, 0x44,
  0x09, 0xde, 0x93, 0x3d, 0x70, 0xa7, 0xea, 0x3e, 0x73, 0xa4, 0xe9, 0x47, 0x0a, 0xdd, 0x90, 0xcc,
  0x81, 0x56, 0x1b,
  0xb5, 0xf8, 0x2f, 0x62, 0x97, 0xda, 0x0d, 0x40, 0xee, 0xa3, 0x74, 0x39, 0x65, 0x28, 0xff, 0xb2,
  0x1c, 0x51, 0x86,
  0xcb, 0x21, 0x6c, 0xbb, 0xf6, 0x58, 0x15, 0xc2, 0x8f, 0xd3, 0x9e, 0x49, 0x04, 0xaa, 0xe7, 0x30,
  0x7d, 0x88, 0xc5,
  0x12, 0x5f, 0xf1, 0xbc, 0x6b, 0x26, 0x7a, 0x37, 0xe0, 0xad, 0x03, 0x4e, 0x99, 0xd4, 0x7c, 0x31,
  0xe6, 0xab, 0x05,
  0x48, 0x9f, 0xd2, 0x8e, 0xc3, 0x14, 0x59, 0xf7, 0xba, 0x6d, 0x20, 0xd5, 0x98, 0x4f, 0x02, 0xac,
  0xe1, 0x36, 0x7b,
  0x27, 0x6a, 0xbd, 0xf0, 0x5e, 0x13, 0xc4, 0x89, 0x63, 0x2e, 0xf9, 0xb4, 0x1a, 0x57, 0x80, 0xcd,
  0x91, 0xdc, 0x0b,
  0x46, 0xe8, 0xa5, 0x72, 0x3f, 0xca, 0x87, 0x50, 0x1d, 0xb3, 0xfe, 0x29, 0x64, 0x38, 0x75, 0xa2,
  0xef, 0x41, 0x0c,
  0xdb, 0x96, 0x42, 0x0f, 0xd8, 0x95, 0x3b, 0x76, 0xa1, 0xec, 0xb0, 0xfd, 0x2a, 0x67, 0xc9, 0x84,
  0x53, 0x1e, 0xeb,
  0xa6, 0x71, 0x3c, 0x92, 0xdf, 0x08, 0x45, 0x19, 0x54, 0x83, 0xce, 0x60, 0x2d, 0xfa, 0xb7, 0x5d,
  0x10, 0xc7, 0x8a,
  0x24, 0x69, 0xbe, 0xf3, 0xaf, 0xe2, 0x35, 0x78, 0xd6, 0x9b, 0x4c, 0x01, 0xf4, 0xb9, 0x6e, 0x23,
  0x8d, 0xc0, 0x17,
  0x5a, 0x06, 0x4b, 0x9c, 0xd1, 0x7f, 0x32, 0xe5, 0xa8]
    crc = 0
    for byte in data:
        crc = crc_table[crc ^ byte]
    return crc

def check_packet_crc(data):
    if len(data) < struct.calcsize('<BBHHHHB') + POINTS_PER_PACKET * 3 + 1:
        return False
    computed_crc = cal_crc8(data[:-1])  # Exclude the last byte (CRC)
    received_crc = data[-1]
    return computed_crc == received_crc

def process_lidar_data(packet):
    data_string = ""
    step = (packet.end_angle - packet.start_angle) / 11.0
    for i in range(POINTS_PER_PACKET):
        angle_degrees = packet.start_angle + step * i
        angle_degrees = angle_degrees / 100 % 360 
        angle_radians = math.radians(angle_degrees)
        
        if 300 < packet.points[i].distance < 12000:
            x = packet.points[i].distance * math.cos(angle_radians)
            y = packet.points[i].distance * math.sin(angle_radians)
            data_string += f"Point {i}: Angle = {angle_degrees:.4f}, Distance = {packet.points[i].distance}\n"
            #print(f"degrees angle: {angle_degrees}, radians angle: {angle_radians}")
    return data_string

def get_lidar_data_from_server():
    try:
        response = requests.get(f'http://{esp32_ip}/lidar')
        if response.status_code == 200:
            # hex_data = response.text
            # binary_data = binascii.unhexlify(hex_data)
            # print(binary_data)
            # process_lidar_data(binary_data)
            hex_data = response.content.decode()  # Assuming the content is a hexadecimal string
            return bytes.fromhex(hex_data)  # Convert to binary
    except Exception as e:
            
    #     else:
    #         print("Failed to retrieve data. Status code:", response.status_code)
    # except Exception as e:
        print()
        
# def get_lidar_data_from_server(esp32_ip, max_retries=3, retry_delay=1):
#     retries = 0
#     while retries < max_retries:
#         try:
#             response = requests.get(f'http://{esp32_ip}/lidar')
#             if response.status_code == 200:
#                 hex_data = response.content.decode()  # Assuming the content is a hexadecimal string
#                 return bytes.fromhex(hex_data)  # Convert to binary            
#             else:
#                 print(f"Failed to get data. Status code: {response.status_code}")
#         except Exception as e:
#             print(f"Error occurred: {e}")
#             time.sleep(retry_delay)  # Wait before retrying
#             retries += 1
#     return None

# def get_lidar_data_from_server( max_retries=3, retry_delay=1):
#     retries = 0
#     # while retries < max_retries:
#     try:
#         with requests.get(f'http://{esp32_ip}/lidar', stream=True) as response:
#             if response.status_code == 200:
#                 for chunk in response.iter_content(chunk_size=1024):
#                     if chunk:  # Filter out keep-alive chunks
#                         hex_data = chunk.decode()  # Assuming each chunk is a hexadecimal string
#                         yield bytes.fromhex(hex_data)  # Convert to binary and yield
#             else:
#                 print(f"Failed to get data. Status code: {response.status_code}")
                
#     except Exception as e:
#         print(f"Error occurred: {e}")
#         time.sleep(retry_delay)  # Wait before retrying
#         retries += 1
#         print(f"retries: {retries}")



In [14]:

## turn on/off lidar unit
def toggle_relay(state):
    try:
        response = requests.get(f'http://{esp32_ip}/toggleRelay?state={state}')
        if response.status_code == 200:
            print(f"Relay state set to {state}")
        else:
            print(f"Failed to set relay state: {response.status_code}")
    except Exception as e:
        print(f"Error occurred: {e}")
        
#turn on wifi
def update_wifi_settings(new_ssid, new_password):
    url = 'http://192.168.4.1/setup'
    data = {'ssid': new_ssid, 'password': new_password}
    print(data)
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    try:
        response = requests.post(url, data=data, headers=headers)
        print("Response Text:", response.text)

        if response.status_code == 200:
            print("Wi-Fi settings updated successfully.")
        else:
            print(f"Failed to update Wi-Fi settings. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error occurred: {e}")
        print("Exception Type:", e.__class__.__name__)
        
def plot_lidar_data(lidar_data_string, ax, *, hdisplay):
    # Clear existing lines
    ax.clear()

    lines = lidar_data_string.split('\n')
    regex = r'Point (\d+): Angle = ([-\d.]+), Distance = ([-\d.]+)'

    for line in lines:
        match = re.search(regex, line)
        if match:
            _, angle, distance = match.groups()
            angles.append(float(angle))
            distances.append(float(distance))
    # Set labels and limits
    ax.set_xlabel('Angle (Radians)')
    ax.set_ylabel('Distance')
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)

    # Convert angles to radians and plot
    angles_radians = np.radians(angles)
    ax.scatter(angles_radians, distances, color='red', s=5)

    # Update the display
    hdisplay.update(fig)



In [15]:
# new_ssid = input("The LAN before Time")
# new_password = input("Jbl?8042")
#update_wifi_settings('The LAN before Time', 'Jbl?8042')


In [28]:
toggle_relay("on") # Turn relay on

Relay state set to on


In [31]:
toggle_relay("off") # Turn relay off

Relay state set to off


In [35]:
# Main loop
angles = []
distances = []


# Create figure and axis for plotting
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
hdisplay = display.display("", display_id=True)

packet_buffer = bytearray()

while True:
    byte_data = get_lidar_data_from_server()

    if byte_data is not None:
        packet_buffer.extend(byte_data)

        if len(packet_buffer) >= PACKET_SIZE:
            if check_packet_crc(packet_buffer):
                # print("Packet passed CRC check.")
                # print(f"Packet Buffer: {packet_buffer}")

                # Ensure to parse the full packet
                packet = parse_lidar_data(packet_buffer)

                if packet:
                    # print(f"Packet Details: {packet}")
                    lidar_data_string = process_lidar_data(packet)
                    
                    plot_lidar_data(lidar_data_string, ax, hdisplay=hdisplay)

                else:
                    print("Failed to parse the packet.")
            
            packet_buffer.clear()
        
        time.sleep(1)  # Adjust delay as needed

    plt.close(fig)


''

KeyboardInterrupt: 